In [1]:
#train model from panel prepared in previous module
#start with naive model for each asset

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit

In [3]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [4]:
data_folder  = "../DataWork/"
panel_fname  = data_folder + "panel.csv"  #panel features file to load
target_fname = data_folder + "target.csv" #target file to load

In [5]:
GAP = 41 #purged days, number of trading days to skip between train to test to prevent leakage

In [6]:
#work with multiindex to allow selection of subset of features based on stock before flattening - TBD
dat = pd.read_csv(panel_fname, header=[0,1], index_col=0)
dat.index = pd.to_datetime(dat.index, format = "%Y-%m-%d")
dat.index.name = 'Date'
dat.tail()

Features       Ret20                                                    \
Stocks          ABBV       ACN       AEP       AIZ      ALLE      AMAT   
Date                                                                     
2022-02-28  0.076479 -0.112304  0.011447  0.111029 -0.069237 -0.027425   
2022-03-01  0.075135 -0.126739  0.029519  0.091906 -0.045782 -0.065333   
2022-03-02  0.076028 -0.121542  0.023703  0.102984 -0.023659 -0.053553   
2022-03-03  0.066522 -0.080103  0.049849  0.120926  0.013686 -0.042151   
2022-03-04  0.068087 -0.100862  0.085920  0.105476 -0.016573 -0.073471   

Features                                                                \
Stocks           AMP      AMZN       AVB       AVY       AXP       BDX   
Date                                                                     
2022-02-28 -0.011418  0.026323 -0.023364 -0.153437  0.078681  0.065266   
2022-03-01 -0.095415 -0.000341 -0.043043 -0.208266 -0.030312  0.069259   
2022-03-02 -0.073791  0.009516 -0.044492 -0.150508 -0.021127  0.059921   
2022-03-03 -0.043044  0.063164 -0.028828 -0.115730 -0.022808  0.008318   
2022-03-04 -0.086781 -0.079166  0.000162 -0.117999 -0.071937  0.024144   

Features                                                                \
Stocks          BF-B       BMY        BR      CARR       CDW        CE   
Date                                                                     
2022-02-28 -0.033170  0.056619 -0.085243 -0.060520 -0.088885 -0.107053   
2022-03-01 -0.046245  0.051424 -0.046933 -0.073405 -0.108578 -0.145642   
2022-03-02 -0.053466  0.060625 -0.040471 -0.056280 -0.108815 -0.098633   
2022-03-03  0.019935  0.075723 -0.014185 -0.007816 -0.098509 -0.095035   
2022-03-04  0.041423  0.065077 -0.016414 -0.018495 -0.111104 -0.093541   

Features                                                                \
Stocks          CHTR       CNC       CNP       COP      CTAS       CZR   
Date                                                                     
2022-02-28  0.014124  0.060625 -0.029921  0.073062 -0.039750  0.100502   
2022-03-01 -0.003609  0.077490 -0.037333  0.062540 -0.037421  0.026623   
2022-03-02 -0.049958  0.071713 -0.015072  0.066324 -0.024552  0.105928   
2022-03-03 -0.083015  0.039089  0.009534  0.074516 -0.014940  0.085909   
2022-03-04 -0.101896  0.065802  0.045025  0.092509 -0.004986 -0.000381   

Features                                                                \
Stocks            DG       DPZ       DRE       DXC       EWA       EWC   
Date                                                                     
2022-02-28 -0.049860 -0.050616 -0.081205  0.123382  0.051622  0.000785   
2022-03-01 -0.039355 -0.058612 -0.073746 -0.022997  0.029802 -0.025663   
2022-03-02 -0.006212 -0.065586 -0.094267  0.029537  0.055558 -0.006717   
2022-03-03  0.004470 -0.097234 -0.077238 -0.107878  0.057400  0.006787   
2022-03-04  0.041715 -0.057214 -0.044069 -0.183155  0.069368  0.003900   

Features                                                                \
Stocks           EWG       EWH       EWJ       EWL       EWQ       EWT   
Date                                                                     
2022-02-28 -0.095998 -0.044413 -0.017950 -0.028582 -0.069955 -0.015061   
2022-03-01 -0.141172 -0.056517 -0.032020 -0.053379 -0.121336 -0.023611   
2022-03-02 -0.136184 -0.064930 -0.043816 -0.047063 -0.107725 -0.014945   
2022-03-03 -0.158839 -0.058968 -0.027687 -0.039989 -0.123241 -0.014817   
2022-03-04 -0.211254 -0.085841 -0.048494 -0.062045 -0.179813 -0.031669   

Features                                                                \
Stocks           EWU       EWY       EWZ        FB       FTV      GOOG   
Date                                                                     
2022-02-28 -0.011873  0.000276  0.039379 -0.395033 -0.084527 -0.005968   
2022-03-01 -0.037752 -0.020628  0.022749 -0.449574 -0.107503 -0.027280   
2022-03-02 -0.025177 -0.027168  0.061913 -0.439586 -0.108252 -0.094027  

In [7]:
tgt = pd.read_csv(target_fname, header=[0,1], index_col=0)
tgt.index = pd.to_datetime(tgt.index, format = "%Y-%m-%d")
tgt.index.name = 'Date'
tgt.head()

Features   Target                                                             \
Stocks       ABBV       ACN       AEP       AIZ ALLE      AMAT AMP      AMZN   
Date                                                                           
2004-12-31    NaN -0.035819  0.026155  0.062798  NaN -0.072760 NaN -0.024456   
2005-01-03    NaN -0.014515  0.045958  0.071942  NaN -0.025287 NaN -0.046905   
2005-01-04    NaN  0.003102  0.050629  0.096216  NaN -0.014652 NaN -0.006189   
2005-01-05    NaN -0.006650  0.061660  0.104255  NaN -0.003706 NaN -0.155628   
2005-01-06    NaN  0.012121  0.062204  0.092931  NaN  0.041964 NaN -0.139080   

Features                                                                    \
Stocks           AVB       AVY       AXP       BDX      BF-B       BMY  BR   
Date                                                                         
2004-12-31 -0.117982  0.001999 -0.052881 -0.002644 -0.009287 -0.077733 NaN   
2005-01-03 -0.091420  0.012095  0.017094  0.027992  0.008290 -0.060948 NaN   
2005-01-04 -0.066307  0.032873  0.014812  0.070773  0.019285 -0.035608 NaN   
2005-01-05 -0.019177  0.033612  0.017661  0.076557  0.029005 -0.039075 NaN   
2005-01-06 -0.026057  0.037855  0.030190  0.073849  0.044443 -0.025784 NaN   

Features                                                                      \
Stocks     CARR CDW  CE CHTR       CNC       CNP       COP      CTAS CZR  DG   
Date                                                                           
2004-12-31  NaN NaN NaN  NaN  0.168410 -0.004435  0.066386 -0.008242 NaN NaN   
2005-01-03  NaN NaN NaN  NaN  0.151806  0.034655  0.117638  0.017000 NaN NaN   
2005-01-04  NaN NaN NaN  NaN  0.142983  0.061108  0.129154  0.054871 NaN NaN   
2005-01-05  NaN NaN NaN  NaN  0.182082  0.084683  0.129827  0.054786 NaN NaN   
2005-01-06  NaN NaN NaN  NaN  0.191211  0.101543  0.119291  0.051890 NaN NaN   

Features                                                                \
Stocks           DPZ       DRE       DXC       EWA       EWC       EWG   
Date                                                                     
2004-12-31 -0.064988 -0.093262 -0.089967  0.007754 -0.046768 -0.048388   
2005-01-03 -0.060553 -0.075391 -0.072946  0.031084 -0.027237 -0.030654   
2005-01-04 -0.052490 -0.045946 -0.060831  0.061714  0.006597 -0.007177   
2005-01-05 -0.048216 -0.007214 -0.057007  0.045147  0.017528 -0.016071   
2005-01-06 -0.014011  0.017656 -0.032987  0.053841  0.021232 -0.008276   

Features                                                                \
Stocks           EWH       EWJ       EWL       EWQ       EWT       EWU   
Date                                                                     
2004-12-31 -0.048294 -0.030686 -0.037919 -0.015615 -0.023491 -0.024624   
2005-01-03 -0.040822 -0.013060 -0.035528 -0.013880 -0.007611 -0.006697   
2005-01-04 -0.018967  0.007512 -0.003610  0.013161  0.031399  0.019080   
2005-01-05 -0.002624 -0.005677 -0.010263  0.004259  0.023953  0.015173   
2005-01-06  0.005177 -0.001889 -0.008434  0.004696  0.045732  0.021301   

Features                                                                  \
Stocks           EWY       EWZ  FB FTV      GOOG       GPC GSG       HIG   
Date                                                                       
2004-12-31  0.057139 -0.031980 NaN NaN  0.014573 -0.040057 NaN -0.029577   
2005-01-03  0.068646 -0.004164 NaN NaN -0.054802 -0.021317 NaN -0.000291   
2005-01-04  0.090569  0.054448 NaN NaN  0.057250  0.004847 NaN  0.011399   
2005-01-05  0.085447  0.073169 NaN NaN  0.085865  0.025183 NaN  0.022765   
2005-01-06  0.122491  0.099452 NaN NaN  0.080520  0.040934 NaN  0.045270   

Features                                                                    \
Stocks     HIGH.L       HST HYG IAU ICLN IEAA.L       IEF IEFM.L IEMG IEUS   
Date                                                                         
2004-12-31    NaN -0.078118 NaN NaN  NaN    NaN  0.008673    NaN

In [8]:
tgt.shape

(4394, 100)

In [9]:
N_ASSETS = tgt.shape[1]

In [10]:
features = ["_".join(reversed(a)) for a in dat.columns.to_flat_index()]

In [11]:
def get_Xy(df, tgt):
    """return data filtered for rows with valid target"""
    X = np.array(df.loc[tgt.dropna().index])
    y = tgt.dropna().values
    return X, y

In [12]:
def train_stock(asset = 0):
    X, y = get_Xy(dat,tgt.iloc[:,asset])
    if len(y) - 5 * len(y)//6 - GAP > 0:
        nsplit = 5
    elif len(y) - 3 * len(y)//4 - GAP > 0:
        nsplit = 3
    else:
        raise ValueError(f"Not enough samples even for 3 splits with {len(y)} samples and {GAP} gaps")
    preds = np.zeros(len(y)) #to hold predictions
    cv = TimeSeriesSplit(n_splits=nsplit, max_train_size = len(y)//nsplit, gap = GAP)
    fold_scores = []
    models = []
    feat_imp =[]
    for i, (train_idx, val_idx) in enumerate(cv.split(X)):
        lgbm = lgb.LGBMRegressor()
        X_train, X_val = X[train_idx,:], X[val_idx,:]
        y_train, y_val = y[train_idx], y[val_idx]
        print(f'Fold {i+1:2} Train length: {len(train_idx)} Test length: {len(val_idx)}')
        lgbm = lgbm.fit(X_train,y_train, eval_set = [(X_val, y_val)], 
                        callbacks = [lgb.early_stopping(10), lgb.log_evaluation(100)])
        pred_val = lgbm.predict(X_val)
        models.append(lgbm)
        fold_scores.append(lgbm.best_score_['valid_0']['l2'])
        feat_imp.append(lgbm.feature_importances_)
        preds[val_idx] += pred_val
    avg_feat_imp = np.mean(feat_imp,axis=0)
    return preds, models, avg_feat_imp, fold_scores

In [13]:
asset_preds = {}
asset_models = {}
asset_feat_imp = {}
asset_fold_scores = {}

In [14]:
%%time
for i in range(N_ASSETS):
    print(f"***TRAIN asset {i+1} {tgt.columns.levels[1][i]}***")
    asset_preds[i], asset_models[i], asset_feat_imp[i], asset_fold_scores[i] = train_stock(i)

***TRAIN asset 1 ABBV***
Fold  1 Train length: 334 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00629768
Fold  2 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's l2: 0.00162177
Fold  3 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00856717
Fold  4 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00737213
Fold  5 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00401729
***TRAIN asset 2 ACN***
Fold  1 Train length: 667 Test length: 707
Training until validation scores don't improve for 10 rounds
Early s

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00297107
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's l2: 0.00297739
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00682464
***TRAIN asset 11 AXP***
Fold  1 Train length: 667 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.0256503
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00346448
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00387817
Fold  4 Train length: 848 Test length: 707
Training until validation sco

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0120108
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00670799
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00551176
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00831238
***TRAIN asset 21 CNP***
Fold  1 Train length: 667 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00971428
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00226356
Fold  3 Tra

Early stopping, best iteration is:
[5]	valid_0's l2: 0.00611418
***TRAIN asset 30 EWC***
Fold  1 Train length: 667 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0107485
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00242955
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00228126
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00114893
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	valid_0's l2: 0.0052686
***TRAIN asset 31 EWG***
Fold  1 Train length: 667 Test length: 707
Train

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00903399
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00763101
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0153433
***TRAIN asset 40 FB***
Fold  1 Train length: 363 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00454862
Fold  2 Train length: 479 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00302331
Fold  3 Train length: 479 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00548909
Fold  4 Train length: 479 Test length: 399
Training until validation scor

...

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00335058
***TRAIN asset 59 IUMO.L***
Fold  1 Train length: 181 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[24]	valid_0's l2: 0.000963668
Fold  2 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0022436
Fold  3 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.0072552
Fold  4 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00293269
Fold  5 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0025827
***TRAIN asset 60 IUVL.L***
Fold  1 Train length: 181 Test length: 22

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00475948
Fold  5 Train length: 536 Test length: 446
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0039868
***TRAIN asset 69 MVEU.L***
Fold  1 Train length: 344 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00185049
Fold  2 Train length: 457 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000849826
Fold  3 Train length: 457 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's l2: 0.000525221
Fold  4 Train length: 457 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00427469
Fold  

Early stopping, best iteration is:
[27]	valid_0's l2: 0.0031444
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00392156
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00990728
***TRAIN asset 79 SEGA.L***
Fold  1 Train length: 489 Test length: 527
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[6]	valid_0's l2: 0.000224159
Fold  2 Train length: 633 Test length: 527
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000166227
Fold  3 Train length: 633 Test length: 527
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's l2: 8.87258e-05
Fold  4 Train length: 633 Test length: 527
Training until valida

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's l2: 0.00242632
Fold  2 Train length: 327 Test length: 272
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00872416
Fold  3 Train length: 327 Test length: 272
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00596703
Fold  4 Train length: 327 Test length: 272
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0227814
Fold  5 Train length: 327 Test length: 272
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00546933
***TRAIN asset 90 XLB***
Fold  1 Train length: 667 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00956662
Fold  2 Tra

Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00253752
***TRAIN asset 99 XLY***
Fold  1 Train length: 667 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00839704
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00242978
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[6]	valid_0's l2: 0.00131515
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00121409
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration 

### Evaluate performance of signals created from trained models

In [15]:
#assume buy for positive prediction and sell for negative
eval_return = np.zeros(N_ASSETS)
eval_last_ret20 = np.zeros(N_ASSETS)
eval_benchmark_ret = np.zeros(N_ASSETS)
for i in range(N_ASSETS):
    actual_rets = tgt.iloc[:,i].dropna().values
    last_ret20_sig = np.nan_to_num(np.sign(dat.loc[tgt.iloc[:,i].dropna().index]['Ret20'].iloc[:,i].values))
    buy_signal = np.sign(asset_preds[i])
    always_buy = np.ones_like(actual_rets)
    eval_last_ret20[i] = np.sum(last_ret20_sig * actual_rets)
    eval_return[i] = sum(buy_signal * actual_rets)
    eval_benchmark_ret[i] = sum(always_buy * actual_rets)

In [16]:
eval_scores = np.zeros(N_ASSETS)
eval_std    = np.zeros(N_ASSETS)
for i in range(N_ASSETS):
    eval_scores[i] = np.mean(asset_fold_scores[i])*100
    eval_std[i]    = np.std(asset_fold_scores[i]) *100

In [17]:
eval_score_df = pd.DataFrame({'MSE Val Score': eval_scores,
                              'MSE Val Score Folds Stdev' : eval_std,
                              'Total Val Return': eval_return,
                              'Total Val LastRet20 Return': eval_last_ret20,
                              'Benchmark BH Val Return': eval_benchmark_ret}, index = dat.columns.levels[1])
eval_score_df

,MSE Val Score,MSE Val Score Folds Stdev,Total Val Return,Total Val LastRet20 Return,Benchmark BH Val Return
Stocks,,,,,
ABBV,0.557521,0.248030,18.582193,-0.282258,34.176040
ACN,0.383701,0.143253,45.851196,1.735003,55.066196
AEP,0.272120,0.131444,20.870826,6.880463,31.797020
AIZ,0.961304,1.061396,5.929501,10.085291,38.546105
ALLE,0.425975,0.289394,24.067577,-5.192243,20.980055
AMAT,0.887140,0.222891,22.637630,-6.885663,46.056601
AMP,1.195175,0.918707,20.643368,15.812962,47.442548
AMZN,0.829765,0.485849,58.827492,9.161708,83.584128
AVB,0.565177,0.507191,23.210803,-11.204706,35.767863


In [18]:
pd.set_option("display.precision",4)
eval_score_df.describe()

,MSE Val Score,MSE Val Score Folds Stdev,Total Val Return,Total Val LastRet20 Return,Benchmark BH Val Return
count,100.0000,100.0000,100.0000,100.0000,100.0000
mean,0.7030,0.6075,12.1859,3.5499,26.2187
std,0.9470,1.2005,16.6980,13.2150,19.2260
min,0.0012,0.0009,-32.0027,-34.9999,-38.2026
25%,0.2835,0.1554,0.8433,-4.1074,13.3401
50%,0.4318,0.2693,11.8612,2.6268,26.1915
75%,0.7299,0.5054,23.0506,9.1747,37.3831
max,5.6183,7.6471,58.8275,52.8295,83.5841


In [19]:
asset_fname = data_folder + "assets_eval_" + pd.to_datetime('today').strftime('%Y_%m_%d') + ".csv"

In [20]:
assets = pd.read_csv(data_folder + "assets.csv", index_col=0)
assets

,class,symbol,name,GICS_sector/ETF_type,GICS_industry/ETF_subtype,FirstTrade,Splits
id,,,,,,,
1,Stock,ABBV,AbbVie,Health Care,Pharmaceuticals,2013-01-02,0
2,Stock,ACN,Accenture,Information Technology,IT Consulting & Other Services,2004-12-31,0
3,Stock,AEP,American Electric Power,Utilities,Electric Utilities,2004-12-31,0
4,Stock,AIZ,Assurant,Financials,Multi-line Insurance,2004-12-31,0
5,Stock,ALLE,Allegion,Industrials,Building Products,2013-11-18,0
6,Stock,AMAT,Applied Materials,Information Technology,Semiconductor Equipment,2004-12-31,0
7,Stock,AMP,Ameriprise Financial,Financials,Asset Management & Custody Banks,2005-09-15,0
8,Stock,AMZN,Amazon,Consumer Discretionary,Internet & Direct Marketing Retail,2004-12-31,0
9,Stock,AVB,AvalonBay Communities,Real Estate,Residential REITs,2004-12-31,0


In [21]:
assets = assets.merge(eval_score_df, left_on='symbol',right_on='Stocks')
assets

,class,symbol,name,GICS_sector/ETF_type,GICS_industry/ETF_subtype,FirstTrade,Splits,MSE Val Score,MSE Val Score Folds Stdev,Total Val Return,Total Val LastRet20 Return,Benchmark BH Val Return
0,Stock,ABBV,AbbVie,Health Care,Pharmaceuticals,2013-01-02,0,0.5575,0.2480,18.5822,-0.2823,34.1760
1,Stock,ACN,Accenture,Information Technology,IT Consulting & Other Services,2004-12-31,0,0.3837,0.1433,45.8512,1.7350,55.0662
2,Stock,AEP,American Electric Power,Utilities,Electric Utilities,2004-12-31,0,0.2721,0.1314,20.8708,6.8805,31.7970
3,Stock,AIZ,Assurant,Financials,Multi-line Insurance,2004-12-31,0,0.9613,1.0614,5.9295,10.0853,38.5461
4,Stock,ALLE,Allegion,Industrials,Building Products,2013-11-18,0,0.4260,0.2894,24.0676,-5.1922,20.9801
5,Stock,AMAT,Applied Materials,Information Technology,Semiconductor Equipment,2004-12-31,0,0.8871,0.2229,22.6376,-6.8857,46.0566
6,Stock,AMP,Ameriprise Financial,Financials,Asset Management & Custody Banks,2005-09-15,0,1.1952,0.9187,20.6434,15.8130,47.4425
7,Stock,AMZN,Amazon,Consumer Discretionary,Internet & Direct Marketing Retail,2004-12-31,0,0.8298,0.4858,58.8275,9.1617,83.5841
8,Stock,AVB,AvalonBay Communities,Real Estate,Residential REITs,2004-12-31,0,0.5652,0.5072,23.2108,-11.2047,35.7679
9,Stock,AVY,Avery Dennison,Materials,Paper Packaging,2004-12-31,0,0.6878,0.4513,32.6016,-15.0514,29.7665


In [22]:
assets.to_csv(asset_fname, index = False)

In [23]:
pd.reset_option("display.precision")

### Predict next value for each asset and create predictions file

In [30]:
features_to_predict = dat.iloc[-1,:].copy().values.reshape(1,-1) #temporary, use last row of data for prediction
preds = np.zeros(tgt.shape[1])
for i in range(tgt.shape[1]):
    models = asset_models[i]
    preds[i] = np.mean(np.array([model.predict(features_to_predict) for model in models]),axis=0)

In [31]:
pd.DataFrame(preds).describe()

,0
count,100.000000
mean,0.008928
std,0.013603
min,-0.043693
25%,0.003773
50%,0.007735
75%,0.013465
max,0.089452


In [32]:
preds_df = pd.DataFrame({'Prediction':preds}, index = dat.columns.levels[1])
preds_df

,Prediction
Stocks,
ABBV,0.020246
ACN,0.010690
AEP,0.011774
AIZ,0.005801
ALLE,0.021437
AMAT,0.014777
AMP,0.011320
AMZN,0.030448
AVB,0.025454


In [33]:
preds_df.to_csv(data_folder + "return_predictions.csv")